# 1. 事前にハイパーパラメータのチューニングを行います

## パラメータチューニング手法
- grid search
- random
- bayse optimzation
※ チューニングには時間がかかります

In [1]:
# チューニング関数への入力
class TrainingConfig:
    csv_path = './sample_data/train_T1.csv' # csvのパス
    task = 'regression' # regression or clastaring
    tuning_method = 'random-search' # チューニングメソッド
    tadk_id = 'T1_pred'
    mode = 'regression'

In [ ]:
from train import train
conf = TrainingConfig
train(conf)

tuning start
X data range will be extract by slice...
shape of x::: (281, 2047)
shape of y::: (281,)
sample of data loaded as X:::[0 0 0 ... 0 0 0]
sample of data loaded as Y:::2.0106
------------First Validation------------
step1 : -0.08645501351104612
step2 : -0.0947016004491407
step3 : -0.042312282614157336
----------------------------------------


 12%|███████████████▏                                                                                                         | 1/8 [00:06<00:48,  6.87s/it]

validation score ::: [-3.66050355 -0.11146106 -0.12140237 -0.10095249 -0.06885404 -0.06695412
 -0.07790129 -0.07559798 -0.06795654 -0.07448963]


 25%|██████████████████████████████▎                                                                                          | 2/8 [00:11<00:33,  5.62s/it]

validation score ::: [-0.18136049 -0.05375263 -0.05879554 -0.0648095  -0.06114965 -0.06810722
 -0.06422743 -0.06234578 -0.06976935 -0.06663503 -0.07448963]


...

 88%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▉               | 7/8 [04:46<00:46, 46.29s/it]

validation score ::: [-0.06132109 -0.06047256 -0.06357972 -0.06843529 -0.06789275 -0.07448963
 -0.06687416 -0.07393    -0.07385229 -0.0749379 ]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [07:03<00:00, 53.00s/it]

validation score ::: [-0.07448963 -0.07454924 -0.07470379 -0.07587375 -0.07486175 -0.08346431
 -0.07862083 -0.09662094]



/Users/poclab_m1_air_2022/.pyenv/versions/3.8.13/lib/python3.8/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 768 is smaller than n_iter=1000. Running 768 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [ ]:
# 2. 予測と評価

In [ ]:
from xgb import xgb_pred_val